<h1> Implementing the ReAct agent without using langgraph framework </h1>
<li>it uses "Thought - Action - Observation" pattern</li>
<li>As sample Add,Multiple methos is implemented</li>




![Screenshot 2024-08-21 at 12.45.43 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbac0b4a2c1e5e02f3e78b_agent2.png)

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import re
import httpx


from langchain_groq import ChatGroq

In [3]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# _set_env("OPENAI_API_KEY")
_set_env("GROQ_API_KEY")

In [4]:
### tools

def calculate(what):
    return eval(what)

def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]




In [34]:
### Prompt Template & LLM

system_template = """You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.
Answer: the final answer to the original question

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Other:
e.g:Other: prompt for the llm to define next action
Runs a llm reasoning and results the response of the llm

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:
# Example1
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris

# Example2
Question: What is 3 * 12 ?
Thought: I should calculate the result of 3 * 12
Action: calculate: 3 * 12
PAUSE


You will be called again with this:

Observation: 36
Answer: 36

Remember:
1. CRITICAL - when your perform multiple task ensure results of all the task is taken in to account
2. Action must be one of the below value ["calculate","wikipedia","Other"]

""".strip()

human_template = """Question: {question}"""

llm = ChatGroq(model="llama3-8b-8192",temperature=0)

In [35]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
}

In [36]:
def parse_output(output):
    # print("Entered Parser - "+output)
    action_match = re.search(r'Action: (.*)', output)
    final_answer_match = re.search(r'Answer: (.*)', output)

    action = action_match.group(1).strip() if action_match else None
    final_answer = final_answer_match.group(1).strip() if final_answer_match else None

    return action, final_answer

In [39]:
### run the agent  - Agent execution is done untill the Final answer is constructed


def run_react_agent(question):
    max_iterations = 10  # Prevent infinite loops
    messages = []
    # Initialize the messages with the system prompt
    messages.append(SystemMessage(content=system_template))
    human_message = human_template.format(question=question)
    messages.append(HumanMessage(content=human_message))
    print(messages[-1].pretty_print(),"\n ===================")
    for _ in range(max_iterations):
        response = llm.invoke(messages)
        messages.append(AIMessage(content=response.content))
        print(messages[-1].pretty_print(),"\n ===================")
        actions,final_answer = parse_output(response.content)
        if final_answer:
            return "Answer: {}".format(final_answer)

        if actions:
            action = ""
            action_input=""
            try:
                action, action_input = actions.split(":")
                print(" -- running {} {}".format(action, action_input))
                if action not in known_actions:
                    raise Exception("Unknown action ") 
                observation = known_actions[action](action_input)
                messages.append(HumanMessage(content=f"Observation: {observation}"))
                print(messages[-1].pretty_print(),"\n ===================")
            except Exception:
                messages.append(HumanMessage(content=f"Observation: {action_input}"))
                print(messages[-1].pretty_print(),"\n ===================")
        else:
            return ""
        print("loop in again")


In [12]:
# query = "What is 4 * 12?"
# print(run_react_agent(query))

In [19]:
query = "What is 4 * 12 and add 3 and divide the result by 2?"
print(run_react_agent(query))

================================ System Message ================================

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.
Answer: the final answer to the original question

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Other:
e.g:Other: Summarize the the Observations
Runs a llm reasoning and results the response of the llm

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:
# Example1
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Actio

In [40]:
query = "What is the sum of 3 and 5 and Explain speciality of India "
print(run_react_agent(query))

================================ Human Message =================================

Question: What is the sum of 3 and 5 and Explain speciality of India 
None 
================================== Ai Message ==================================

Thought: I should calculate the sum of 3 and 5, and then look up India on Wikipedia to find out about its specialties.

Action: calculate: 3 + 5

PAUSE
None 
 -- running calculate  3 + 5
================================ Human Message =================================

Observation: 8
None 
loop in again
================================== Ai Message ==================================

Thought: Now that I have the sum, I should look up India on Wikipedia to find out about its specialties.

Action: wikipedia: India

PAUSE
None 
 -- running wikipedia  India
================================ Human Message =================================

Observation: <span class="searchmatch">India</span>, officially the Republic of <span class="searchmatch">India</span> 

In [42]:
query = """Answer below question 
    1: What is the sum of 3 and 5 
    2. How to improve my communication 
    3. Explain speciality of India"""
print(run_react_agent(query))

================================ Human Message =================================

Question: Answer below question 
    1: What is the sum of 3 and 5 
    2. How to improve my communication 
    3. Explain speciality of India
None 
================================== Ai Message ==================================

Thought: I should calculate the sum of 3 and 5, and also consider the question about improving communication and explaining the specialty of India.

Action: calculate: 3 + 5

PAUSE
None 
 -- running calculate  3 + 5
================================ Human Message =================================

Observation: 8
None 
loop in again
================================== Ai Message ==================================

Thought: Now that I have the sum, I should think about how to improve communication. I'll consider searching for information on effective communication techniques.

Action: Other: prompt for the llm to define next action

PAUSE
None 
 -- running Other  prompt for the llm 

In [43]:
query = """Add 3 and 4, then multiply by 2, and finally divide by 5"""
print(run_react_agent(query))

================================ Human Message =================================

Question: Add 3 and 4, then multiply by 2, and finally divide by 5
None 
================================== Ai Message ==================================

Thought: I need to perform a series of operations to get the final result. First, I'll add 3 and 4, then multiply the result by 2, and finally divide by 5.

Action: calculate: (3 + 4) * 2 / 5

PAUSE
None 
 -- running calculate  (3 + 4) * 2 / 5
================================ Human Message =================================

Observation: 2.8
None 
loop in again
================================== Ai Message ==================================

Answer: The result of the operations is 2.8.
None 
Answer: The result of the operations is 2.8.
